In [1]:
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import  WebDriverWait
import pandas as pd 
import time, os, requests, threading, ast
from bs4 import BeautifulSoup
from selenium import webdriver


In [2]:
def scrape_all_vehicles(start_path):

    os.makedirs("html", exist_ok=True)
    os.makedirs("xlsx", exist_ok=True)


    options = ChromeOptions()
    prefs = {
        "download.default_directory": os.path.abspath("xlsx"),
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True,
    }
    options.add_experimental_option("prefs", prefs)

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    driver.maximize_window()
    driver.get(start_path)

    # ✅ Step 1: Login (if login form exists)
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "loginForm.Email"))
        ).send_keys("fourbrotherstrading@icloud.com")

        WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.ID, "loginForm.Password"))
        ).send_keys("Muhssan7865")

        WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.XPATH, "//input[@value='Login']"))
        ).click()

        print("✅ Logged in successfully.")
        time.sleep(3)
    except:
        print("ℹ️ Login not required or already logged in.")


    try:
        download_link = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "a.js-export-to-excel"))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", download_link)
        time.sleep(1)
        download_link.click()
        print("✅ Excel download started...")

        timeout = 30
        for _ in range(timeout):
            files = [f for f in os.listdir("xlsx") if f.endswith(".xls") or f.endswith(".xlsx")]
            if files:
                print(f"✅ File downloaded: {files[0]}")
                break
            time.sleep(1)
        else:
            print("⚠️ Excel download timeout.")
    except Exception as e:
        print("❌ Could not click download link:", e)
    global_event_time = "N/A"  # default value

    try:
        event_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//div[@class="ed__date__inner"]'))
        )

        time_ = event_box.find_element(By.XPATH, './/div[@class="time"]').text.strip()
        global_event_time = time_
        print("✅ Event time found:", global_event_time)

    except Exception as e:
        print("⚠️ Event time not found:", e)

    print(global_event_time)

    try:
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(2)

        # Wait until at least one vehicle title link is loaded
        first_vehicle_link = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "a.car__title"))
        )

        # Scroll to it
        driver.execute_script("arguments[0].scrollIntoView({block:'center'});", first_vehicle_link)
        time.sleep(1)

        clicked = False
        for attempt in range(3):
            try:
                first_vehicle_link.click()
                clicked = True
                print(f"✅ Clicked first vehicle title normally (try {attempt+1})")
                break
            except Exception:
                try:
                    driver.execute_script("arguments[0].click();", first_vehicle_link)
                    clicked = True
                    print(f"✅ Clicked first vehicle title via JS (try {attempt+1})")
                    break
                except Exception:
                    print(f"⚠️ Click attempt {attempt+1} failed, retrying...")
                    driver.execute_script("window.scrollBy(0, 200)")
                    time.sleep(2)
                    first_vehicle_link = driver.find_element(By.CSS_SELECTOR, "a.car__title")

        if not clicked:
            raise Exception("Failed to click first vehicle title after 3 attempts.")

        # Wait until the vehicle detail page opens (URL check)
        WebDriverWait(driver, 20).until(
            EC.url_contains("/vehicle-detail/")
        )
        print("✅ Vehicle detail page opened successfully.")

    except Exception as e:
        print("❌ Could not click vehicle title or open detail page:", e)
        driver.save_screenshot("first_vehicle_click_error.png")



    time.sleep(2)


    vehicle_count = 1
    all_regs = []

    while True:
        try:
            # Expand vehicle info (if collapsed)
            try:
                info_block = driver.find_element(By.CLASS_NAME, "vdb__title")
                info_block.click()
                time.sleep(1)
            except:
                pass

            # Extract registration number
            reg_no = "unknown"
            try:
                items = driver.find_elements(By.CLASS_NAME, "vdb__item")
                for item in items:
                    label = item.find_element(By.CLASS_NAME, "vdb__label").text.strip()
                    if label == "REG NO":
                        reg_no = item.find_element(By.CLASS_NAME, "vdb__value").text.strip()
                        break
            except:
                pass

            # Save page HTML
            html_path = os.path.join("html", f"{reg_no}.html")
            with open(html_path, "w", encoding="utf-8") as f:
                f.write(driver.page_source)
            print(f"{vehicle_count}. Saved HTML as {reg_no}.html")
            all_regs.append(reg_no)
            vehicle_count += 1

            # Go to next vehicle
            try:
                next_btn = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.ID, "nextLink"))
                )
                if "disabled" in next_btn.get_attribute("class"):
                    print("✅ All vehicles scraped.")
                    break
                driver.execute_script("arguments[0].scrollIntoView({block:'center'});", next_btn)
                time.sleep(1)
                driver.execute_script("arguments[0].click();", next_btn)
                time.sleep(2)
            except:
                print("✅ No more next button — done.")
                break

        except Exception as e:
            print(f"⚠️ Error while scraping vehicle {vehicle_count}: {e}")
            break


    driver.quit()
    print("\nAll registration numbers collected:")
    print(all_regs)
    return all_regs, global_event_time



if __name__ == "__main__":
    path = "https://www.manheim.co.uk/catalogues-and-events/listing/46447?locationId=5&locationName=Bristol"
    all_regs, event_time = scrape_all_vehicles(path)

✅ Logged in successfully.
✅ Excel download started...
✅ File downloaded: Bristol Lane 1 - Sytner Mon Mtrs Heritage.xlsx
✅ Event time found: 10:00
10:00
✅ Clicked first vehicle title normally (try 1)
✅ Vehicle detail page opened successfully.
1. Saved HTML as SJ15EKR.html
2. Saved HTML as V20BJD.html
3. Saved HTML as BJ13CNA.html
4. Saved HTML as WJ66TYP.html
5. Saved HTML as HJ14DBX.html
6. Saved HTML as DV09CZN.html
7. Saved HTML as VK07WYA.html
8. Saved HTML as KV19DLZ.html
9. Saved HTML as EN56TBO.html
10. Saved HTML as LS15HBX.html
11. Saved HTML as FY57GWA.html
12. Saved HTML as FB07BVH.html
13. Saved HTML as WA68HGO.html
14. Saved HTML as J171NAM.html
15. Saved HTML as KV09ZVX.html
16. Saved HTML as GD66NYG.html
17. Saved HTML as MM66LKD.html
18. Saved HTML as VX14UDE.html
19. Saved HTML as WG13OZA.html
20. Saved HTML as VO69WSX.html
21. Saved HTML as SJ11VGU.html
22. Saved HTML as PJ63BVD.html
23. Saved HTML as MW12FRV.html
24. Saved HTML as LW16BDV.html
25. Saved HTML as RX12BK

In [3]:
def delete_unknown_html(folder="html", filename="unknown.html"):
    file_path = os.path.join(folder, filename)
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"Deleted: {file_path}")
    else:
        print(f"File not found: {file_path}")

# Call the function
delete_unknown_html()

File not found: html\unknown.html


In [ ]:
import os
import csv
from bs4 import BeautifulSoup
import re

html_folder = "html"
output_csv = "manheim_data.csv"

with open(output_csv, "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    # Write header
    writer.writerow([
        "Reg", "Title","Auction name", "Transmission", "Fuel Type", "CC", "Doors",
        "Cap Clean", "Cap Average", "Cap Retail", "Grade","Body Type","sc",
        "Year", "D.O.R", "Mileage", "BASE COLOUR", "COLOUR",
        "COLOUR TYPE", "INTERIOR COLOUR", "TRIM TYPE", "Seats",
        "Keys", "V5", "VAT Status", "T Loss", "Non Runner", "IMPORTED","Vendor","Center","Features","Additional Information","Buy Now Price",
        "Grade Link","Images","INSPECTION REPORT","Former Keepers","NO OF SERVICES","Last Service","Last Service mileage","Service History",
        "DVSA Mileage","SERVICED BY","SERVICE RECORD TYPE","Start Time","MOT Expiry Date"
    ])

    for filename in os.listdir(html_folder):
        if filename.endswith(".html"):
            file_path = os.path.join(html_folder, filename)
            reg_no = filename.replace(".html", "")
            
            with open(file_path, "r", encoding="utf-8") as f:
                soup = BeautifulSoup(f, "html.parser")
            
    
            title_tag = soup.select_one("div.vehicle-details__description h1.vehicle-details__title")
            titleText = title_tag.get_text(strip=True) if title_tag else ""
            
            aucName_tag = soup.select_one("span.bold.widget-event-name")
            aucNameText = aucName_tag.get_text(strip=True) if aucName_tag else ""
            
            # Transmission / Fuel / CC / Doors
            transmission = soup.select_one("div.vehicle-details__properties")
            transmission_gettext = transmission.get_text(strip=True) if transmission else ""
            parts = transmission_gettext.split(" - ")

            
            transmission_val = parts[0] if len(parts) > 0 else ""
            fuel_type = parts[1] if len(parts) > 1 else ""

            # Extract CC
            cc_match = re.search(r'([\d\.]+)L', parts[2]) if len(parts) > 2 else None
            cc = cc_match.group(1) if cc_match else ""

            # Extract Doors
            door_match = re.search(r'(\d+)\s*Door', parts[3]) if len(parts) > 3 else None
            door = door_match.group(1) if door_match else ""

            # Extract Body Type (everything after "Door")
            body_type_match = re.search(r'Door\s*(.*)', parts[3]) if len(parts) > 3 else None
            body_type = body_type_match.group(1).strip() if body_type_match else ""


            cap_clean = cap_average = cap_retail = ""
            blocks = soup.find_all("div", class_="vehicle-details__block vdb")
            for block in blocks:
                title_div = block.find("div", class_="vdb__title")
                if title_div and "Pricing Information" in title_div.get_text():
                    dl_tag = block.find("dl", class_="vdb__dl")
                    if dl_tag:
                        dt_dd_pairs = dl_tag.find_all(["dt","dd"])
                        cap_dict = {}
                        for i in range(0, len(dt_dd_pairs), 2):
                            key = dt_dd_pairs[i].get_text(strip=True)
                            value_tag = dt_dd_pairs[i+1].find("span", class_="vdb__value")
                            value = value_tag.get_text(strip=True) if value_tag else ""
                            cap_dict[key] = value

                        cap_clean = cap_dict.get("Cap clean", "")
                        cap_average = cap_dict.get("Cap average", "")
                        cap_retail = cap_dict.get("Cap retail", "")
                    break

            # Grade
            gradeElement = soup.select_one("a.button_grade")
            grade_number = ""
            if gradeElement:
                gradeText = gradeElement.get_text(strip=True)
                match = re.search(r'\d+', gradeText)
                if match:
                    grade_number = match.group(0)
                gradeLink = f"https://www.manheim.co.uk{gradeElement.get("href", "")}"
            else:
                gradeText = ""
                gradeLink = ""
                
            
         
                
            sc_element = soup.select_one("a.car__button.car__button_grey")
            sc_text = sc_element.get_text(strip=True) if sc_element else ""

            vendor_text = next(
                (
                    item.find("a").get_text(strip=True)
                    for item in soup.find_all("div", class_="vdb__item")
                    if item.find(class_="vdb__label") and item.find(class_="vdb__label").get_text(strip=True) == "Vendor"
                ),
                ""
            )
            center_text = next(
                (
                    item.find("a").get_text(strip=True)
                    for item in soup.find_all("div", class_="vdb__item")
                    if item.find(class_="vdb__label") and item.find(class_="vdb__label").get_text(strip=True) == "Vehicle Location"
                ),
                ""
            )
            
            inspection_report_text = ""
            inspection_report_link = ""
            keeper_text = ""
            mot_expiry = ""
            num_services = ""
            last_service_date = ""
            odometer_last_service = ""

            for item in soup.find_all("div", class_="vdb__item"):
                label = item.find(class_="vdb__label")
                if not label:
                    continue

                label_text = label.get_text(strip=True)
                value_tag = item.find(class_="vdb__value")
                value_text = value_tag.get_text(strip=True) if value_tag else ""

                # 🔹 Inspection Report
                if label_text == "Inspection Report":
                    a_tag = value_tag.find("a") if value_tag else None
                    if a_tag:
                        inspection_report_text = a_tag.get_text(strip=True)
                        inspection_report_link = a_tag.get("href", "")

                # 🔹 Registered Keepers
                elif label_text == "Registered Keepers":
                    keeper_text = value_text

                # 🔹 MOT Expiry Date
                elif label_text == "MOT Expiry Date":
                    mot_expiry = value_text

                # (Optional: you can also extract these)
                elif label_text == "No Of Services":
                    num_services = value_text

                elif label_text == "Date of Last Service":
                    last_service_date = value_text

                elif label_text == "Odometer at Last Service":
                    odometer_last_service = value_text


            # Example of storing in details dict
            inspection_report_text  = inspection_report_text
            inspection_report_link = inspection_report_link
            keeper_text = keeper_text
            mot_expiry = mot_expiry
            num_services = num_services
            last_service_date = last_service_date
            odometer_last_service = odometer_last_service
            
            buynoepriceElement = soup.select_one("span.current-price.current-price_buy-now")
            buynowText = buynoepriceElement.get_text(strip=True) if buynoepriceElement else ""
            
            blockss = soup.select("div.vehicle-details__block.vehicle-details__clear.vdb")

            AdditionalInfo = ""

            for block in blockss:
                title_element = block.select_one("a.vdb__title.js-dropdown-onmobile")
                if title_element and "Additional Information" in title_element.get_text(strip=True):
                    # Found the correct section
                    info_items = block.select("div.vdb__content_text li")
                    AdditionalInfo = ", ".join(li.get_text(strip=True) for li in info_items if li.get_text(strip=True))
                    break  # stop after first match

            # Vehicle Information block
            vehicle_info = {}
            vehicle_blocks = soup.find_all("div", class_="vehicle-details__block vdb")
            for block in vehicle_blocks:
                title_div = block.find("a", class_="vdb__title")
                if title_div and "Vehicle Information" in title_div.get_text():
                    items = block.find_all("div", class_="vdb__item")
                    for item in items:
                        try:
                            label = item.find(class_="vdb__label").get_text(strip=True)
                            value = item.find(class_="vdb__value").get_text(strip=True)
                            vehicle_info[label] = value
                        except:
                            continue
                    break 
                
            spec_block = soup.select_one("div.vehicle-details__block.vdb")
            features = ""

            blocks = soup.select("div.vehicle-details__block.vdb")

            for block in blocks:
                title = block.select_one("a.vdb__title")
                if title and "Specification" in title.get_text(strip=True):
                    feature_items = block.select("ul.vdb__columns li")
                    features = ", ".join(li.get_text(strip=True) for li in feature_items)
                    break   

         
            service_block = soup.select_one("div.vehicle-details__block.vdb.vdb_table")

            no_of_services = 0
            last_service_date = ""
            last_service_mileage = ""
            service_history_dates = []
            dvsa_mileages = []
            serviced_by_list = []
            service_record_type_list = []

            if service_block:
                rows = service_block.select("div.vdb__row:not(.vdb__row_header)")
                no_of_services = len(rows)

                for row in rows:
                    cols = row.select("div.vdb__td")
                    if len(cols) >= 4:
                        date = cols[0].get_text(strip=True)
                        mileage = cols[1].get_text(strip=True)
                        serviced_by = cols[2].get_text(strip=True)
                        record_type = cols[3].get_text(strip=True)

                        service_history_dates.append(date)
                        dvsa_mileages.append(mileage)
                        serviced_by_list.append(serviced_by)
                        service_record_type_list.append(record_type)

                # last (most recent) record
                if rows:
                    last_service_date = service_history_dates[-1] if service_history_dates else ""
                    last_service_mileage = dvsa_mileages[-1] if dvsa_mileages else ""

            # Convert lists to strings for CSV
            service_history = ", ".join(service_history_dates)
            dvsa_mileage = ", ".join(dvsa_mileages)
            serviced_by = ", ".join(serviced_by_list)
            service_record_type = ", ".join(service_record_type_list)
            image_block = soup.select_one("div.vehicle-details__gallery div.flexslider ul.slides")
            zoom_images = []

            if image_block:
                for li in image_block.select("li"):
                    a_tag = li.find("a", class_="fancybox-360-link")
                    if a_tag and a_tag.has_attr("imagehigh"):
                        zoom_images.append(a_tag["imagehigh"])

            # Comma-separated string
            zoom_images_str = ", ".join(zoom_images)
            # event_time=""
            writer.writerow([
                reg_no, titleText,aucNameText, transmission_val, fuel_type, cc, door,
                cap_clean, cap_average, cap_retail, grade_number,body_type,sc_text,
                vehicle_info.get("MFR YEAR",""),
                vehicle_info.get("REG DATE",""),
                vehicle_info.get("Odometer",""),
                vehicle_info.get("BASE COLOUR",""),
                vehicle_info.get("COLOUR",""),
                vehicle_info.get("Colour Type",""),
                vehicle_info.get("Interior Colour",""),
                vehicle_info.get("TRIM TYPE",""),
                vehicle_info.get("NO OF SEATS",""),
                vehicle_info.get("NO OF KEYS",""),
                vehicle_info.get("V5",""),
                vehicle_info.get("VAT",""),
                vehicle_info.get("Total Loss",""),
                vehicle_info.get("RUNNER",""),
                vehicle_info.get("IMPORTED",""),
                vendor_text,center_text,
                features,AdditionalInfo,
                buynowText,gradeLink,zoom_images_str,
                inspection_report_link,keeper_text,
                no_of_services,
                last_service_date,
                last_service_mileage,
                service_history,
                dvsa_mileage,
                serviced_by,
                service_record_type,event_time,mot_expiry
            ])
            print(f"Processed {reg_no}: {title}")

print(f"All data saved to {output_csv}")


Processed BJ13CNA: <a class="vdb__title js-dropdown-onmobile" href="javascript:;">Specification</a>
Processed BW69XGH: <a class="vdb__title js-dropdown-onmobile" href="javascript:;">Specification</a>
Processed DV09CZN: <a class="vdb__title js-dropdown-onmobile" href="javascript:;">Specification</a>
Processed EN56TBO: <a class="vdb__title js-dropdown-onmobile" href="javascript:;">Specification</a>
Processed FB07BVH: <a class="vdb__title js-dropdown-onmobile" href="javascript:;">Specification</a>
Processed FY57GWA: <a class="vdb__title js-dropdown-onmobile" href="javascript:;">Specification</a>
Processed GD66NYG: <a class="vdb__title js-dropdown-onmobile" href="javascript:;">Specification</a>
Processed GL64LHY: <a class="vdb__title js-dropdown-onmobile" href="javascript:;">Specification</a>
Processed HJ14DBX: <a class="vdb__title js-dropdown-onmobile" href="javascript:;">Specification</a>
Processed HY70ZTW: <a class="vdb__title js-dropdown-onmobile" href="javascript:;">Specification</a>


In [ ]:
import os
import pandas as pd
from glob import glob

folder_path = os.path.join(os.getcwd(), "xlsx")
excel_files = glob(os.path.join(folder_path, "*.xlsx"))

if not excel_files:
    raise FileNotFoundError("⚠️ No Excel files found in 'xlsx' folder!")

latest_file = max(excel_files, key=os.path.getmtime)
print("✅ Latest Excel file found:", latest_file)


def load_excel_or_csv(file_path):
    try:
        df = pd.read_excel(file_path, engine="openpyxl", skiprows=1)
        print("✅ Loaded as Excel successfully (first row skipped).")
        return df
    except Exception as e1:
        print(f"⚠️ Excel load failed ({e1}) — trying CSV...")
        try:
            df = pd.read_csv(file_path, engine="python", skiprows=1)
            print("✅ Loaded as CSV successfully (first row skipped).")
            return df
        except Exception as e2:
            print(f"❌ Both Excel and CSV failed: {e2}")
            return pd.DataFrame()


# --- Load latest Excel or CSV ---
df = load_excel_or_csv(latest_file)

# --- Clean and rename columns ---
df.columns = df.columns.str.strip()

if 'Reg No.' not in df.columns:
    possible_cols = [c for c in df.columns if 'Reg' in c]
    if possible_cols:
        df.rename(columns={possible_cols[0]: 'Reg No.'}, inplace=True)

# --- Add Lot No and Event date if present ---
optional_cols = ['Lot No', 'Event date']
needed_cols = ['Reg No.', 'Manufacturer', 'Model', 'Derivative']

# Add only columns that exist in file
all_needed_cols = [col for col in needed_cols + optional_cols if col in df.columns]

df = df[all_needed_cols]
df.dropna(how='all', inplace=True)

if 'Lot No' in df.columns:
    df['Lot No'] = pd.to_numeric(df['Lot No'], errors='coerce').fillna(0).astype(int)
# --- Temporary CSV for merging ---
temp_csv = "to_merge.csv"
df.to_csv(temp_csv, index=False)

# --- Load main live CSV ---
live_file = "manheim_data.csv"
if not os.path.exists(live_file):
    raise FileNotFoundError(f"⚠️ Live data file '{live_file}' not found!")

top_car_df = pd.read_csv(temp_csv)
top_car_live_cleaned_df = pd.read_csv(live_file)

# Clean column names
top_car_live_cleaned_df.columns = top_car_live_cleaned_df.columns.str.strip()

if 'Reg No.' not in top_car_live_cleaned_df.columns and 'Reg' in top_car_live_cleaned_df.columns:
    top_car_live_cleaned_df.rename(columns={'Reg': 'Reg No.'}, inplace=True)

# --- Merge ---
merged_df = top_car_live_cleaned_df.merge(
    top_car_df,
    on='Reg No.',
    how='left'
)

merged_df.dropna(how='all', inplace=True)
output_path = "manheim_data_merged.csv"
merged_df.to_csv(output_path, index=False)

# Cleanup temp file
if os.path.exists(temp_csv):
    os.remove(temp_csv)

print("🎉 Merge complete! File saved as:", output_path)
print("📊 Columns merged:", ", ".join(all_needed_cols))


✅ Latest Excel file found: d:\shakeeb\3-Manheim\data\xlsx\Bristol Lane 1 - Sytner Mon Mtrs Heritage.xlsx
✅ Loaded as Excel successfully (first row skipped).
🎉 Merge complete! File saved as: manheim_data_merged.csv
📊 Columns merged: Reg No., Manufacturer, Model, Derivative, Lot No, Event date


In [6]:
df = pd.read_csv("manheim_data_merged.csv")
df.rename(columns={
    "Derivative": "Variant",
    "Manufacturer": "Make",
    "Reg No.": "Reg",
    "Lot No": "Lot",
    "Event date": "Start Date"
}, inplace=True)

# 2️⃣ Process Mileage and Mileage warranted
df['Mileage warranted'] = df['Mileage'].apply(
    lambda x: x.split('\n')[1] if isinstance(x, str) and '\n' in x else None
)
df['Mileage warranted'] = df['Mileage warranted'].apply(
    lambda x: x.split("(")[1].split(")")[0] if isinstance(x, str) and "(" in x and ")" in x else 'na'
)
df['Mileage'] = df['Mileage'].apply(
    lambda x: x.split(" ")[0] if isinstance(x, str) and " " in x else "na"
)

# 3️⃣ Save cleaned data
df.to_csv("manheim_data_cleaned.csv", index=False)

# 4️⃣ Remove old merged file
if os.path.exists("manheim_data_merged.csv"):
    os.remove("manheim_data_merged.csv")

print("✅ Columns renamed and mileage processed. File saved as 'manheim_data_cleaned.csv'.")


✅ Columns renamed and mileage processed. File saved as 'manheim_data_cleaned.csv'.


In [7]:
from PIL import Image, ImageDraw, ImageFont
def add_watermark_to_image(image_path, text="Sourced from Manheim Auction"):
    try:
        image = Image.open(image_path).convert("RGBA")
        txt_layer = Image.new("RGBA", image.size, (255, 255, 255, 0))
        draw = ImageDraw.Draw(txt_layer)

        # Load font
        try:
            font = ImageFont.truetype("arial.ttf", 16)
        except:
            font = ImageFont.load_default()

        # Calculate text size and position
        margin = 10
        bbox = draw.textbbox((0, 0), text, font=font)
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]
        x = image.width - text_width - margin
        y = image.height - text_height - margin

        # Draw semi-transparent background box
        box_width = text_width + 2 * margin
        box_height = text_height + 2 * margin
        draw.rectangle([x - margin, y - margin, x - margin + box_width, y - margin + box_height],fill=(0, 0, 0, 160))

        # Draw watermark text
        draw.text((x, y), text, font=font, fill=(255, 255, 255, 200))

        # Merge and save
        watermarked = Image.alpha_composite(image, txt_layer).convert("RGB")
        watermarked.save(image_path)
        print(f"Watermark added to {image_path}")

    except Exception as e:
        print(f"Failed to watermark {image_path}: {e}")


In [8]:
file_path = "manheim_data_cleaned.csv"
df = pd.read_csv(file_path)

# 🔹 Ensure column exists
if "Images" in df.columns:
    # Replace ims_tiny with zoom in all URLs
    df["Images"] = df["Images"].astype(str).str.replace(
        "/ims_tiny/", "/zoom/", regex=False
    )

    # 🔹 Save updated file (overwrite)
    df.to_csv(file_path, index=False)
    print("✅ All image URLs updated successfully in:", file_path)
else:
    print("⚠️ 'Images' column not found in the file.")

✅ All image URLs updated successfully in: manheim_data_cleaned.csv


In [9]:
from urllib.parse import urlparse, urljoin


df =pd.read_csv("manheim_data_cleaned.csv")
reg_img = df[["Reg", "Images"]]
report = df[['Reg', 'INSPECTION REPORT']]

def download_reports(data, main_folder="Inspec_report"): 
    

    os.makedirs(main_folder, exist_ok=True)
    

    for index, row in data.iterrows():
        reg_no = row["Reg"] 
        report_urls = row["INSPECTION REPORT"]

        if not report_urls or pd.isna(report_urls):
            print(f"Missing Inspection Report of {reg_no}")

            
        else:
            if not report_urls.startswith(("http://", "https://")): 
                report_urls = urljoin("https://", report_urls) 
            

            parsed_url = urlparse(report_urls)
            

            try:

                response = requests.get(report_urls, stream=True) 
                response.raise_for_status() 
                

                file_name = os.path.basename(parsed_url.path) or f"inspec_repo.pdf"


                file_extension = file_name.split(".")[-1]
                

                if file_extension not in ["pdf", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"inspec_repo.pdf" 
                

                full_file_name = os.path.join(main_folder, f"{reg_no}.pdf")
                

                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024): 
                        f.write(chunk)
                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {report_urls} for {reg_no}: {e}")


def download_images(data, main_folder="Images"): 
    

    os.makedirs(main_folder, exist_ok=True)
    

    for index, row in data.iterrows():
        reg_no = row["Reg"] 
        if pd.isna(row["Images"]) or not isinstance(row["Images"], str) or row["Images"].strip() == "":
            print(f"Skipping {reg_no} due to missing images.")
            continue  

        image_urls = row["Images"].split(", ")
        

        reg_folder = os.path.join(main_folder, reg_no) 
        os.makedirs(reg_folder, exist_ok=True) 
        

        for idx, url in enumerate(image_urls):
            url = url.strip() 
            if not url.startswith(("http://", "https://")): 
                url = urljoin("https://", url) 
            

            parsed_url = urlparse(url)

 
            if not parsed_url.scheme or not parsed_url.netloc:
                print(f"Invalid URL skipped: {url}") 
                continue
            

            try:

                response = requests.get(url, stream=True)
                response.raise_for_status() 

                file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"


                file_extension = file_name.split(".")[-1]
                

                if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"image_{idx + 1}.jpg" 
                

                full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}.jpg")
                

                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)


                add_watermark_to_image(full_file_name)
                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {url} for {reg_no}: {e}")

def start_funcs():
    thread1 = threading.Thread(target=download_reports, args=(report,))
    thread2 = threading.Thread(target=download_images, args=(reg_img,))


    thread1.start()
    thread2.start()
    

    thread1.join()
    thread2.join()

if __name__ == "__main__":
    start_funcs()

Watermark added to Images\BJ13CNA\BJ13CNA_1.jpg
Downloaded: Images\BJ13CNA\BJ13CNA_1.jpg
Watermark added to Images\BJ13CNA\BJ13CNA_2.jpg
Downloaded: Images\BJ13CNA\BJ13CNA_2.jpg
Watermark added to Images\BJ13CNA\BJ13CNA_3.jpg
Downloaded: Images\BJ13CNA\BJ13CNA_3.jpg
Downloaded: Inspec_report\BJ13CNA.pdf
Watermark added to Images\BJ13CNA\BJ13CNA_4.jpg
Downloaded: Images\BJ13CNA\BJ13CNA_4.jpg
Watermark added to Images\BJ13CNA\BJ13CNA_5.jpg
Downloaded: Images\BJ13CNA\BJ13CNA_5.jpg
Downloaded: Inspec_report\BW69XGH.pdf
Watermark added to Images\BJ13CNA\BJ13CNA_6.jpg
Downloaded: Images\BJ13CNA\BJ13CNA_6.jpg
Downloaded: Inspec_report\DV09CZN.pdf
Watermark added to Images\BJ13CNA\BJ13CNA_7.jpg
Downloaded: Images\BJ13CNA\BJ13CNA_7.jpg
Watermark added to Images\BJ13CNA\BJ13CNA_8.jpg
Downloaded: Images\BJ13CNA\BJ13CNA_8.jpg
Downloaded: Inspec_report\EN56TBO.pdf
Watermark added to Images\BJ13CNA\BJ13CNA_9.jpg
Downloaded: Images\BJ13CNA\BJ13CNA_9.jpg
Watermark added to Images\BJ13CNA\BJ13CNA_10.jp

KeyboardInterrupt: 